# Binary Classification
- Algorithms & Methods
- Class Imbalances
- Model Selection

In [1]:
# Importing libraries
import pandas as pd
import numpy as np
from Preprocessing import IncomePreprocess

In [2]:
train_data = pd.read_csv('../data/census_income_learn.csv', header=None)
test_data = pd.read_csv('../data/census_income_test.csv', header=None)

In [3]:
# Preprocessing step
ip  = IncomePreprocess()
df, df_test = ip.label_features(train_data, test_data)
X_train, X_test = ip.preprocess(df, df_test)

In [4]:
X_train.head()

,age,class_of_work,industry_code,occupation_code,education,wage_per_hour,enrolled_in_edu_inst_last_wk,marital_status,major_industry_code,major_occupation_code,...,own_business_or_self_employed,fill_inc_questionnaire_for_veterans_admin,veteran_benefits,weeks_worked_in_year,year,y,has_gains,has_losses,has_stock,origin
0,73,0,0,0,9,0,2,4,14,6,...,0,1,2,0,1,0,0.0,0.0,0.0,1
1,58,3,4,34,10,0,2,5,4,8,...,0,1,2,52,0,0,0.0,0.0,0.0,1
2,18,0,0,0,6,0,1,7,14,6,...,0,1,2,0,1,0,0.0,0.0,0.0,2
3,9,0,0,0,0,0,2,7,14,6,...,0,1,0,0,0,0,0.0,0.0,0.0,1
4,10,0,0,0,0,0,2,7,14,6,...,0,1,0,0,0,0,0.0,0.0,0.0,1
